<a href="https://colab.research.google.com/github/s3965610/COSC2671-A2/blob/asif-recommender-system/Github-recommender-system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from pandas import json_normalize
import pandas as pd
import numpy as np

In [2]:
with open("/content/drive/MyDrive/data/user_repos.json", "r") as read_file:
    user_repos = json.load(read_file)


In [3]:
df = json_normalize(user_repos) 

In [4]:
user_repos_dict = {}
data = []
for i in user_repos:
    temp = {}
    temp['Total Starred'] = user_repos[i]['data']['user']['starredRepositories']['totalCount']
    temp['StarredRepos'] = []
    row = []
    row.append(i)
    row.append(temp['Total Starred'] )
    for node in user_repos[i]['data']['user']['starredRepositories']['nodes']:
        temp['StarredRepos'].append(node['nameWithOwner'])

    row.append(temp['StarredRepos'])

    data.append(row)
    user_repos_dict[i] = temp


In [ ]:
user_repos_dict['glenux']

In [5]:
df = pd.DataFrame.from_dict(user_repos_dict)

In [6]:
numpy_array = np.array(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [7]:
df = pd.DataFrame(numpy_array, columns=['User', 'Total Starred', 
                      'Repos'])

In [11]:
df.head(5)

,User,Total Starred,Repos
0,01885326842,1,[novuhq/novu]
1,0x00evil,1441,"[atom/atom, greatfire/wiki, rails-sqlserver/ac..."
2,0xWDG,240,"[nygard/class-dump, 0xWDG/tvOS.js, apple/swift..."
3,1016135097,25,"[sumanth-ch/OA-Services, JStumpp/awesome-andro..."
4,11ph22il,497,"[dustin/twitty-twister, reddit-archive/reddit-..."


In [8]:
mergedlist = []
for i in df['Repos'].dropna():
    mergedlist.extend(i)
repos = sorted(set(mergedlist))

In [9]:
def jaccard_similarity(list1, list2):
    union = len(set(list1 + list2))
    list3 = [value for value in list1 if value in list2]
    intersection = len(list3)
    if(union == 0):
        return 0
    return (intersection * 1.0)/union



In [10]:
similarity_matrix = {}
for i in user_repos_dict.keys():
    similarity_matrix[i] = {}
    for j in user_repos_dict.keys():
        l1 = user_repos_dict[i]['StarredRepos']
        l2 = user_repos_dict[j]['StarredRepos']
        similarity_matrix[i][j] = jaccard_similarity(l1, l2)


In [12]:
import heapq
k_keys_sorted = heapq.nlargest(5, similarity_matrix['0x00evil'])


In [13]:
k_keys_sorted

['zzragida', 'zzcclp', 'zsy619', 'zplanet', 'zoozoogao']

In [ ]:
similarity_matrix['0x00evil']

In [17]:
def getTopKUsers(k, user):
    k_keys_sorted = heapq.nlargest(5, similarity_matrix['0x00evil'])
    return k_keys_sorted


In [18]:
def mostCommonRepo(users, repo_dict):
    repos = {}
    for u in users:
        for repo in repo_dict[u]['StarredRepos']:
            if repo in repo_dict.keys():
                repos[repo] += 1
            else: 
                repos[repo] = 1
    return repos

In [19]:
test = mostCommonRepo(k_keys_sorted, user_repos_dict)

In [20]:
test

{'JakeWharton/ActionBarSherlock': 1,
 'castorflex/FlipImageView': 1,
 'takimafr/androidkickstartr': 1,
 'nostra13/Android-Universal-Image-Loader': 1,
 'keyboardsurfer/Crouton': 1,
 'square/dagger': 1,
 'square/retrofit': 1,
 'square/android-times-square': 1,
 'ragunathjawahar/instant-adapter': 1,
 'chrisbanes/Android-BitmapCache': 1,
 'Hanul/UJS': 1,
 'dotnet/corefx': 1,
 'dotnet-foundation/foundation': 1,
 'ricardojmendez/LibNoise.Unity': 1,
 'strongloop/loopback': 1,
 'google/seesaw': 1,
 'bayandin/awesome-awesomeness': 1,
 'fffaraz/awesome-cpp': 1,
 'avelino/awesome-go': 1,
 'akullpp/awesome-java': 1,
 'commaai/research': 1,
 'commaai/neo': 1,
 'commaai/openpilot': 1,
 'loom/loom': 1,
 'nhn/socket.io-client-unity3d': 1,
 'Cysharp/MagicOnion': 1,
 'cockroachdb/cockroach': 1,
 'line/armeria': 1,
 'realm/realm-dotnet': 1,
 'grpc/grpc': 1,
 'couchbase/couchbase-lite-net': 1,
 'BBuzzArt/react-native-infinite': 1,
 'openfaas/faas': 1,
 'apache/openwhisk': 1,
 'sschmid/Entitas': 1,
 'scout